In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from xgboost import XGBClassifier
from scipy.stats import kurtosis, skew
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
%cd Z:\PPMI_Data\Excels\CollaborativeFiltering
NonMot = pd.read_csv('Feats45_unCategAge_APPRDX.csv')
NonMot1 = NonMot.drop(['PATNO', 'Patient_ID', 'Age'], axis = 1)
NonMot1['APPRDX'] = NonMot1['APPRDX'].replace([1], 'Patient')
NonMot1['APPRDX'] = NonMot1['APPRDX'].replace([2], 'Healthy')
NonMot1.head(2)

Z:\PPMI_Data\Excels\CollaborativeFiltering


,APPRDX,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,Patient,1,0,12.16,7,1,1,1,1,0,...,57,1,0,45,59,47.5,110,204,17,1
1,Patient,1,1,7.76,6,1,1,0,0,0,...,36,1,3,40,39,52.0,27,52,9,0


# Naive Based

In [25]:

output = []
for i in range(1, 101):
    print(i)
    kfold = KFold(n_splits=10, shuffle=True, random_state=i)
    accuracies = []
    for train_index, test_index in kfold.split(NonMot1.drop('APPRDX', axis=1), NonMot1['APPRDX']):
        train_data, test_data = NonMot1.iloc[train_index].drop('APPRDX', axis=1), NonMot1.iloc[test_index].drop('APPRDX', axis=1)
        train_labels, test_labels = NonMot1.iloc[train_index]['APPRDX'], NonMot1.iloc[test_index]['APPRDX']
    
        nb_model = GaussianNB()
        nb_model.fit(train_data, train_labels)
    
        predictions = nb_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        accuracies.append(accuracy)
    mean_accuracy = np.mean(accuracies)
    print(f'Mean accuracy: {mean_accuracy}')
    output.append(mean_accuracy)

np.savetxt("NonMotor_unCateg_100NaiveBayes_10fold.csv", output, delimiter=",")


1
Mean accuracy: 0.716
2
Mean accuracy: 0.72
3
Mean accuracy: 0.72
4
Mean accuracy: 0.704
5
Mean accuracy: 0.7
6
Mean accuracy: 0.72
7
Mean accuracy: 0.712
8
Mean accuracy: 0.712
9
Mean accuracy: 0.712
10
Mean accuracy: 0.732
11
Mean accuracy: 0.716
12
Mean accuracy: 0.716
13
Mean accuracy: 0.708
14
Mean accuracy: 0.716
15
Mean accuracy: 0.712
16
Mean accuracy: 0.696
17
Mean accuracy: 0.708
18
Mean accuracy: 0.696
19
Mean accuracy: 0.716
20
Mean accuracy: 0.7
21
Mean accuracy: 0.7
22
Mean accuracy: 0.728
23
Mean accuracy: 0.72
24
Mean accuracy: 0.708
25
Mean accuracy: 0.688
26
Mean accuracy: 0.708
27
Mean accuracy: 0.716
28
Mean accuracy: 0.704
29
Mean accuracy: 0.708
30
Mean accuracy: 0.716
31
Mean accuracy: 0.7
32
Mean accuracy: 0.704
33
Mean accuracy: 0.708
34
Mean accuracy: 0.708
35
Mean accuracy: 0.716
36
Mean accuracy: 0.712
37
Mean accuracy: 0.708
38
Mean accuracy: 0.704
39
Mean accuracy: 0.704
40
Mean accuracy: 0.712
41
Mean accuracy: 0.716
42
Mean accuracy: 0.728
43
Mean accur

In [12]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                                                NonMot1['APPRDX'], test_size=0.2)
        
        nb_model = GaussianNB()
        nb_model.fit(train_data, train_labels)
        
        predictions = nb_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_NaiveBased.csv", output, delimiter=",")
        joblib.dump(nb_model, f"{col}_NaiveBased_model.joblib")


Accuracy for column Anxiety: 0.66
Accuracy for column Apathy: 0.42
Accuracy for column Benton: 0.68
Accuracy for column Clock: 0.62
Accuracy for column Cognition: 0.62
Accuracy for column COGSTATE: 0.5
Accuracy for column Constipate: 0.7
Accuracy for column Depress: 0.5
Accuracy for column DopaDefic: 0.66
Accuracy for column Education: 0.56
Accuracy for column Epworth: 0.7
Accuracy for column Fatigue: 0.6
Accuracy for column Geriatric_Depression: 0.6
Accuracy for column Hallucination: 0.7
Accuracy for column Hand: 0.56
Accuracy for column Hopkins: 0.72
Accuracy for column Hopkins_Recog: 0.7
Accuracy for column Impulsive_CompulsiveBehavior: 0.62
Accuracy for column Impulsive_ICD: 0.74
Accuracy for column LetterNumber: 0.56
Accuracy for column Lexical_Fluency: 0.7
Accuracy for column LightHead: 0.42
Accuracy for column Modif_Boston: 0.76
Accuracy for column Montreal_Cognitive: 0.58
Accuracy for column Pain: 0.7
Accuracy for column REM_AwakeDream: 0.76
Accuracy for column REM_AwakeProblem

# Bagging

In [26]:

output = []
for i in range(1, 101):
    print(f'Run {i}')
    data = NonMot1.drop('APPRDX', axis=1)
    labels = NonMot1['APPRDX']
    kf = KFold(n_splits=10, shuffle=True)
    accuracies = []
    for train_index, test_index in kf.split(data):
        train_data, test_data = data.iloc[train_index], data.iloc[test_index]
        train_labels, test_labels = labels.iloc[train_index], labels.iloc[test_index]
        base_model = DecisionTreeClassifier()
        bagging_model = BaggingClassifier(base_estimator=base_model, n_estimators=10, max_samples=0.8, max_features=0.8)
        bagging_model.fit(train_data, train_labels)
        predictions = bagging_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        accuracies.append(accuracy)
    mean_accuracy = np.mean(accuracies)
    print(f'Mean accuracy: {mean_accuracy}')
    output.append(mean_accuracy)

np.savetxt("NonMotor_unCateg_100bagging.csv", output, delimiter=",")


Run 1
Mean accuracy: 0.8160000000000001
Run 2
Mean accuracy: 0.796
Run 3
Mean accuracy: 0.796
Run 4
Mean accuracy: 0.808
Run 5
Mean accuracy: 0.8160000000000001
Run 6
Mean accuracy: 0.792
Run 7
Mean accuracy: 0.792
Run 8
Mean accuracy: 0.792
Run 9
Mean accuracy: 0.804
Run 10
Mean accuracy: 0.784
Run 11
Mean accuracy: 0.8160000000000001
Run 12
Mean accuracy: 0.8200000000000001
Run 13
Mean accuracy: 0.8
Run 14
Mean accuracy: 0.8440000000000001
Run 15
Mean accuracy: 0.808
Run 16
Mean accuracy: 0.8560000000000001
Run 17
Mean accuracy: 0.828
Run 18
Mean accuracy: 0.8360000000000001
Run 19
Mean accuracy: 0.8240000000000001
Run 20
Mean accuracy: 0.78
Run 21
Mean accuracy: 0.788
Run 22
Mean accuracy: 0.8240000000000001
Run 23
Mean accuracy: 0.796
Run 24
Mean accuracy: 0.8200000000000001
Run 25
Mean accuracy: 0.812
Run 26
Mean accuracy: 0.808
Run 27
Mean accuracy: 0.7999999999999999
Run 28
Mean accuracy: 0.808
Run 29
Mean accuracy: 0.8240000000000001
Run 30
Mean accuracy: 0.8280000000000001
Run

In [19]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        X = NonMot1[[col]]
        y = NonMot1['APPRDX']
        
        base_model = DecisionTreeClassifier()
        bagging_model = BaggingClassifier(base_estimator=base_model, n_estimators=10, max_samples=0.8, max_features=0.8)
        bagging_model.fit(X,y)
        # 10-fold cross-validation
        scores = cross_val_score(bagging_model, X, y, cv=10)
        print(f'Accuracy for column {col}: {np.mean(scores)}')
        output.append(np.mean(scores))

        np.savetxt(f"{col}_Bag.csv", output, delimiter=",")
        joblib.dump(bagging_model, f"{col}_bag_model.joblib")


Accuracy for column Anxiety: 0.6559999999999999
Accuracy for column Apathy: 0.6559999999999999
Accuracy for column Benton: 0.572
Accuracy for column Clock: 0.636
Accuracy for column Cognition: 0.6559999999999999
Accuracy for column COGSTATE: 0.6559999999999999
Accuracy for column Constipate: 0.6559999999999999
Accuracy for column Depress: 0.6559999999999999
Accuracy for column DopaDefic: 0.6479999999999999
Accuracy for column Education: 0.6279999999999999
Accuracy for column Epworth: 0.6479999999999999
Accuracy for column Fatigue: 0.624
Accuracy for column Geriatric_Depression: 0.6439999999999999
Accuracy for column Hallucination: 0.6479999999999999
Accuracy for column Hand: 0.6559999999999999
Accuracy for column Hopkins: 0.6359999999999999
Accuracy for column Hopkins_Recog: 0.6359999999999999
Accuracy for column Impulsive_CompulsiveBehavior: 0.64
Accuracy for column Impulsive_ICD: 0.6519999999999999
Accuracy for column LetterNumber: 0.612
Accuracy for column Lexical_Fluency: 0.5720000

# Decision Tree

In [27]:

output = []
for i in range(1, 101):
    print(i)
    data = NonMot1.drop('APPRDX', axis=1)
    labels = NonMot1['APPRDX']
    
    dt_model = DecisionTreeClassifier()
    accuracy_scores = cross_val_score(dt_model, data, labels, cv=10)
    
    mean_accuracy = accuracy_scores.mean()
    print(f'Accuracy: {mean_accuracy}')
    output.append(mean_accuracy)

np.savetxt("NonMotor_unCateg_100DecisionTree.csv", output, delimiter=",")


1
Accuracy: 0.764
2
Accuracy: 0.732
3
Accuracy: 0.74
4
Accuracy: 0.736
5
Accuracy: 0.748
6
Accuracy: 0.76
7
Accuracy: 0.74
8
Accuracy: 0.756
9
Accuracy: 0.752
10
Accuracy: 0.748
11
Accuracy: 0.748
12
Accuracy: 0.756
13
Accuracy: 0.732
14
Accuracy: 0.756
15
Accuracy: 0.732
16
Accuracy: 0.732
17
Accuracy: 0.752
18
Accuracy: 0.736
19
Accuracy: 0.748
20
Accuracy: 0.732
21
Accuracy: 0.752
22
Accuracy: 0.768
23
Accuracy: 0.752
24
Accuracy: 0.732
25
Accuracy: 0.732
26
Accuracy: 0.748
27
Accuracy: 0.736
28
Accuracy: 0.728
29
Accuracy: 0.728
30
Accuracy: 0.756
31
Accuracy: 0.748
32
Accuracy: 0.748
33
Accuracy: 0.744
34
Accuracy: 0.752
35
Accuracy: 0.752
36
Accuracy: 0.744
37
Accuracy: 0.768
38
Accuracy: 0.728
39
Accuracy: 0.728
40
Accuracy: 0.744
41
Accuracy: 0.74
42
Accuracy: 0.748
43
Accuracy: 0.732
44
Accuracy: 0.748
45
Accuracy: 0.736
46
Accuracy: 0.748
47
Accuracy: 0.752
48
Accuracy: 0.756
49
Accuracy: 0.74
50
Accuracy: 0.732
51
Accuracy: 0.74
52
Accuracy: 0.736
53
Accuracy: 0.744
54
Accur

In [8]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                                                NonMot1['APPRDX'], test_size=0.2)
        
        dt_model = DecisionTreeClassifier()
        dt_model.fit(train_data, train_labels)
        
        predictions = dt_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_DecTree.csv", output, delimiter=",")
        joblib.dump(dt_model, f"{col}_DecTree_model.joblib")


Accuracy for column Anxiety: 0.66
Accuracy for column Apathy: 0.74
Accuracy for column Benton: 0.56
Accuracy for column Clock: 0.64
Accuracy for column Cognition: 0.6
Accuracy for column COGSTATE: 0.56
Accuracy for column Constipate: 0.64
Accuracy for column Depress: 0.54
Accuracy for column DopaDefic: 0.64
Accuracy for column Education: 0.54
Accuracy for column Epworth: 0.62
Accuracy for column Fatigue: 0.5
Accuracy for column Geriatric_Depression: 0.62
Accuracy for column Hallucination: 0.66
Accuracy for column Hand: 0.5
Accuracy for column Hopkins: 0.54
Accuracy for column Hopkins_Recog: 0.48
Accuracy for column Impulsive_CompulsiveBehavior: 0.62
Accuracy for column Impulsive_ICD: 0.62
Accuracy for column LetterNumber: 0.66
Accuracy for column Lexical_Fluency: 0.5
Accuracy for column LightHead: 0.6
Accuracy for column Modif_Boston: 0.5
Accuracy for column Montreal_Cognitive: 0.6
Accuracy for column Pain: 0.64
Accuracy for column REM_AwakeDream: 0.74
Accuracy for column REM_AwakeProb

# Random Forest

In [18]:

output = []
for i in range(1, 101):
    print(i)
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators=100, max_depth=5)
    scores = cross_val_score(random_forest_model, X, y, cv=10)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    output.append(mean_score)

np.savetxt("NonMotor_unCateg_100RandomForest_10CV.csv", output, delimiter=",")


1
Mean accuracy: 0.8200000000000001
2
Mean accuracy: 0.8160000000000001
3
Mean accuracy: 0.8240000000000001
4
Mean accuracy: 0.8240000000000001
5
Mean accuracy: 0.8160000000000001
6
Mean accuracy: 0.8240000000000001
7
Mean accuracy: 0.8320000000000001
8
Mean accuracy: 0.808
9
Mean accuracy: 0.8160000000000001
10
Mean accuracy: 0.8240000000000001
11
Mean accuracy: 0.8360000000000001
12
Mean accuracy: 0.8280000000000001
13
Mean accuracy: 0.8360000000000001
14
Mean accuracy: 0.8240000000000001
15
Mean accuracy: 0.8160000000000001
16
Mean accuracy: 0.8240000000000001
17
Mean accuracy: 0.8320000000000001
18
Mean accuracy: 0.8200000000000001
19
Mean accuracy: 0.8160000000000001
20
Mean accuracy: 0.8360000000000001
21
Mean accuracy: 0.808
22
Mean accuracy: 0.8320000000000001
23
Mean accuracy: 0.8240000000000001
24
Mean accuracy: 0.8200000000000001
25
Mean accuracy: 0.8240000000000001
26
Mean accuracy: 0.8240000000000001
27
Mean accuracy: 0.8360000000000001
28
Mean accuracy: 0.8160000000000001

In [10]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                        NonMot1['APPRDX'], test_size=0.2)
        
        random_forest_model = RandomForestClassifier(n_estimators=100, max_depth=5)
        random_forest_model.fit(train_data, train_labels)
        
        predictions = random_forest_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_RF.csv", output, delimiter=",")
        joblib.dump(random_forest_model, f"{col}_RF_model.joblib")

Accuracy for column Anxiety: 0.62
Accuracy for column Apathy: 0.64
Accuracy for column Benton: 0.64
Accuracy for column Clock: 0.56
Accuracy for column Cognition: 0.68
Accuracy for column COGSTATE: 0.8
Accuracy for column Constipate: 0.62
Accuracy for column Depress: 0.58
Accuracy for column DopaDefic: 0.72
Accuracy for column Education: 0.62
Accuracy for column Epworth: 0.58
Accuracy for column Fatigue: 0.58
Accuracy for column Geriatric_Depression: 0.66
Accuracy for column Hallucination: 0.64
Accuracy for column Hand: 0.52
Accuracy for column Hopkins: 0.7
Accuracy for column Hopkins_Recog: 0.58
Accuracy for column Impulsive_CompulsiveBehavior: 0.64
Accuracy for column Impulsive_ICD: 0.84
Accuracy for column LetterNumber: 0.66
Accuracy for column Lexical_Fluency: 0.6
Accuracy for column LightHead: 0.64
Accuracy for column Modif_Boston: 0.68
Accuracy for column Montreal_Cognitive: 0.68
Accuracy for column Pain: 0.66
Accuracy for column REM_AwakeDream: 0.62
Accuracy for column REM_Awake

# LASSO

In [28]:

output = []
for i in range(1, 101):
    print(i)
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    lasso_model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=500)
    parameters = {'C': [0.1, 1, 10, 100]}
    lasso_model = GridSearchCV(lasso_model, parameters)
    lasso_model.fit(X_train, y_train)
    
    cv_scores = cross_val_score(lasso_model, X, y, cv=10)
    mean_cv_score = np.mean(cv_scores)
    print(f'Mean cross-validation accuracy: {mean_cv_score}')
    
    predictions = lasso_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'Test accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100Lasso.csv", output, delimiter=",")


1
Mean cross-validation accuracy: 0.8
Test accuracy: 0.78
2
Mean cross-validation accuracy: 0.8
Test accuracy: 0.86
3
Mean cross-validation accuracy: 0.8
Test accuracy: 0.8
4
Mean cross-validation accuracy: 0.8
Test accuracy: 0.8
5
Mean cross-validation accuracy: 0.8
Test accuracy: 0.8
6
Mean cross-validation accuracy: 0.8
Test accuracy: 0.8
7
Mean cross-validation accuracy: 0.8
Test accuracy: 0.86
8
Mean cross-validation accuracy: 0.8
Test accuracy: 0.8
9
Mean cross-validation accuracy: 0.8
Test accuracy: 0.84
10
Mean cross-validation accuracy: 0.8
Test accuracy: 0.76
11
Mean cross-validation accuracy: 0.8
Test accuracy: 0.78
12
Mean cross-validation accuracy: 0.8
Test accuracy: 0.72
13
Mean cross-validation accuracy: 0.8
Test accuracy: 0.74
14
Mean cross-validation accuracy: 0.8
Test accuracy: 0.84
15
Mean cross-validation accuracy: 0.8
Test accuracy: 0.66
16
Mean cross-validation accuracy: 0.8
Test accuracy: 0.84
17
Mean cross-validation accuracy: 0.8
Test accuracy: 0.86
18
Mean cro

output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    lasso_model = LogisticRegression(penalty='l1', solver = 'liblinear', max_iter = 500)
    parameters = {'C': [0.1, 1, 10, 100]}
    lasso_model = GridSearchCV(lasso_model, parameters)
    lasso_model.fit(train_data, train_labels)

    
    predictions = lasso_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100Lasso.csv", output, delimiter=",")


In [5]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], NonMot1['APPRDX'], 
                                                                            test_size=0.2)

        lasso_model = LogisticRegression(penalty='l1', solver = 'liblinear', max_iter = 1000)
        parameters = {'C': [0.1, 1, 10, 100]}
        lasso_model = GridSearchCV(lasso_model, parameters)
        lasso_model.fit(train_data, train_labels)

        predictions = lasso_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_Lasso.csv", output, delimiter=",")
        joblib.dump(lasso_model, f"{col}_Lasso_model.joblib")


Accuracy for column Anxiety: 0.64
Accuracy for column Apathy: 0.7
Accuracy for column Benton: 0.56
Accuracy for column Clock: 0.72
Accuracy for column Cognition: 0.54
Accuracy for column COGSTATE: 0.6
Accuracy for column Constipate: 0.76
Accuracy for column Depress: 0.82
Accuracy for column DopaDefic: 0.8
Accuracy for column Education: 0.68
Accuracy for column Epworth: 0.62
Accuracy for column Fatigue: 0.66
Accuracy for column Geriatric_Depression: 0.78
Accuracy for column Hallucination: 0.6
Accuracy for column Hand: 0.58
Accuracy for column Hopkins: 0.62
Accuracy for column Hopkins_Recog: 0.72
Accuracy for column Impulsive_CompulsiveBehavior: 0.62
Accuracy for column Impulsive_ICD: 0.64
Accuracy for column LetterNumber: 0.68
Accuracy for column Lexical_Fluency: 0.62
Accuracy for column LightHead: 0.58
Accuracy for column Modif_Boston: 0.8
Accuracy for column Montreal_Cognitive: 0.7
Accuracy for column Pain: 0.58
Accuracy for column REM_AwakeDream: 0.62
Accuracy for column REM_AwakePro

# Elastic Net

In [30]:
output = []
for i in range(1, 101):
    print(i)
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    enet_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000, l1_ratio=0.5)
    parameters = {'C': [0.1, 1, 10, 100]}
    enet_model = GridSearchCV(enet_model, parameters)
    enet_model.fit(X_train, y_train)
    
    cv_scores = cross_val_score(enet_model, X, y, cv=10)
    mean_cv_score = np.mean(cv_scores)
    print(f'Mean cross-validation accuracy: {mean_cv_score}')
    
    predictions = enet_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'Test accuracy: {accuracy}')
    output.append(accuracy)
    
np.savetxt("NonMotor_unCateg_100ElasticNet.csv", output, delimiter=",")


1
Mean cross-validation accuracy: 0.772
Test accuracy: 0.84
2
Mean cross-validation accuracy: 0.772
Test accuracy: 0.84
3
Mean cross-validation accuracy: 0.772
Test accuracy: 0.82
4
Mean cross-validation accuracy: 0.772
Test accuracy: 0.86
5
Mean cross-validation accuracy: 0.772
Test accuracy: 0.86
6
Mean cross-validation accuracy: 0.772
Test accuracy: 0.86
7
Mean cross-validation accuracy: 0.772
Test accuracy: 0.8
8
Mean cross-validation accuracy: 0.772
Test accuracy: 0.8
9
Mean cross-validation accuracy: 0.772
Test accuracy: 0.84
10
Mean cross-validation accuracy: 0.772
Test accuracy: 0.76
11
Mean cross-validation accuracy: 0.772
Test accuracy: 0.76
12
Mean cross-validation accuracy: 0.772
Test accuracy: 0.88
13
Mean cross-validation accuracy: 0.772
Test accuracy: 0.76
14
Mean cross-validation accuracy: 0.772
Test accuracy: 0.82
15
Mean cross-validation accuracy: 0.772
Test accuracy: 0.86
16
Mean cross-validation accuracy: 0.772
Test accuracy: 0.72
17
Mean cross-validation accuracy: 

In [6]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], NonMot1['APPRDX'], 
                                                                            test_size=0.2)

        enet_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000, l1_ratio=0.5)
        parameters = {'C': [0.1, 1, 10, 100]}
        enet_model = GridSearchCV(enet_model, parameters)
        enet_model.fit(train_data, train_labels)
        
        predictions = enet_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_enet.csv", output, delimiter=",")
        joblib.dump(enet_model, f"{col}_enet_model.joblib")


Accuracy for column Anxiety: 0.62
Accuracy for column Apathy: 0.6
Accuracy for column Benton: 0.62
Accuracy for column Clock: 0.64
Accuracy for column Cognition: 0.66
Accuracy for column COGSTATE: 0.72
Accuracy for column Constipate: 0.74
Accuracy for column Depress: 0.58
Accuracy for column DopaDefic: 0.68
Accuracy for column Education: 0.6
Accuracy for column Epworth: 0.7
Accuracy for column Fatigue: 0.62
Accuracy for column Geriatric_Depression: 0.56
Accuracy for column Hallucination: 0.64
Accuracy for column Hand: 0.66
Accuracy for column Hopkins: 0.68
Accuracy for column Hopkins_Recog: 0.64
Accuracy for column Impulsive_CompulsiveBehavior: 0.68
Accuracy for column Impulsive_ICD: 0.66
Accuracy for column LetterNumber: 0.56
Accuracy for column Lexical_Fluency: 0.66
Accuracy for column LightHead: 0.66
Accuracy for column Modif_Boston: 0.54
Accuracy for column Montreal_Cognitive: 0.66
Accuracy for column Pain: 0.64
Accuracy for column REM_AwakeDream: 0.74
Accuracy for column REM_Awake

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Accuracy for column STAIS: 0.6
Accuracy for column Symbol_Digit: 0.82
Accuracy for column Trail_Making_A: 0.66
Accuracy for column Trail_Making_B: 0.6
Accuracy for column UPSIT: 0.82
Accuracy for column Urine: 0.62


# SGD Classifier

In [29]:

output = []
for i in range(1, 101):
    print(i)
    skf = StratifiedKFold(n_splits=10, shuffle=True)
    accuracies = []
    
    for train_index, test_index in skf.split(NonMot1.drop('APPRDX', axis=1), NonMot1['APPRDX']):
        train_data, test_data = NonMot1.drop('APPRDX', axis=1).iloc[train_index], NonMot1.drop('APPRDX', axis=1).iloc[test_index]
        train_labels, test_labels = NonMot1['APPRDX'].iloc[train_index], NonMot1['APPRDX'].iloc[test_index]

        sgd_model = SGDClassifier(loss='log', max_iter=10000)
        parameters = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1], 'penalty': ['l1', 'l2', 'elasticnet']}
        sgd_model = GridSearchCV(sgd_model, parameters)
        sgd_model.fit(train_data, train_labels)

        predictions = sgd_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        accuracies.append(accuracy)

    avg_accuracy = sum(accuracies)/len(accuracies)
    print(f'Average accuracy: {avg_accuracy}')
    output.append(avg_accuracy)

np.savetxt("NonMotor_unCateg_100sgd.csv", output, delimiter=",")


1
Average accuracy: 0.7879999999999999
2
Average accuracy: 0.7799999999999999
3
Average accuracy: 0.7959999999999999
4


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Average accuracy: 0.7839999999999999
5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Average accuracy: 0.7999999999999999
6


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Average accuracy: 0.7799999999999999
96


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Average accuracy: 0.7839999999999999
97
Average accuracy: 0.8119999999999999
98
Average accuracy: 0.7839999999999999
99
Average accuracy: 0.7799999999999999
100
Average accuracy: 0.7999999999999999


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [4]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                        NonMot1['APPRDX'], test_size=0.2)
        
        sgd_model = SGDClassifier(loss='log', max_iter=10000)
        parameters = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1], 'penalty': ['l1', 'l2', 'elasticnet']}
        sgd_model = GridSearchCV(sgd_model, parameters)
        sgd_model.fit(train_data, train_labels)

        predictions = sgd_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_SGD.csv", output, delimiter=",")
        joblib.dump(sgd_model, f"{col}_sgd_model.joblib")

Accuracy for column Anxiety: 0.74
Accuracy for column Apathy: 0.56
Accuracy for column Benton: 0.36
Accuracy for column Clock: 0.44
Accuracy for column Cognition: 0.56
Accuracy for column COGSTATE: 0.6
Accuracy for column Constipate: 0.72
Accuracy for column Depress: 0.8
Accuracy for column DopaDefic: 0.66
Accuracy for column Education: 0.78
Accuracy for column Epworth: 0.66
Accuracy for column Fatigue: 0.58
Accuracy for column Geriatric_Depression: 0.7
Accuracy for column Hallucination: 0.64
Accuracy for column Hand: 0.46
Accuracy for column Hopkins: 0.68
Accuracy for column Hopkins_Recog: 0.66
Accuracy for column Impulsive_CompulsiveBehavior: 0.68
Accuracy for column Impulsive_ICD: 0.52
Accuracy for column LetterNumber: 0.64
Accuracy for column Lexical_Fluency: 0.3
Accuracy for column LightHead: 0.66
Accuracy for column Modif_Boston: 0.68
Accuracy for column Montreal_Cognitive: 0.64
Accuracy for column Pain: 0.7
Accuracy for column REM_AwakeDream: 0.58
Accuracy for column REM_AwakePr

# SVM 

## SVM Linear


output = []
for i in range(1, 101):
    print(i)
    data = NonMot1.drop('APPRDX', axis=1)
    labels = NonMot1['APPRDX']
    svm_model = SVC(kernel='linear')
    parameters = {'C': [0.01, 0.1, 0.5, 1, 10, 50, 100]}
    svm_model = GridSearchCV(svm_model, parameters)
    scores = cross_val_score(svm_model, data, labels, cv=10)
    accuracy = np.mean(scores)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100svmL.csv", output, delimiter=",")


In [36]:
output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    svm_model = SVC(kernel='linear')
    parameters = {'C': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100]}
    svm_model = GridSearchCV(svm_model, parameters)
    svm_model.fit(train_data, train_labels)
    
    predictions = svm_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100svmL.csv", output, delimiter=",")


1
Accuracy: 0.68
2
Accuracy: 0.82
3
Accuracy: 0.68
4
Accuracy: 0.82
5
Accuracy: 0.72
6
Accuracy: 0.82
7
Accuracy: 0.76
8
Accuracy: 0.74
9
Accuracy: 0.82
10
Accuracy: 0.78
11
Accuracy: 0.84
12
Accuracy: 0.86
13
Accuracy: 0.74
14
Accuracy: 0.82
15
Accuracy: 0.72
16
Accuracy: 0.82
17
Accuracy: 0.7
18
Accuracy: 0.86
19
Accuracy: 0.82
20
Accuracy: 0.78
21
Accuracy: 0.7
22
Accuracy: 0.8
23
Accuracy: 0.78
24
Accuracy: 0.8
25
Accuracy: 0.88
26
Accuracy: 0.82
27
Accuracy: 0.68
28
Accuracy: 0.8
29
Accuracy: 0.78
30
Accuracy: 0.82
31
Accuracy: 0.84
32
Accuracy: 0.86
33
Accuracy: 0.82
34
Accuracy: 0.76
35
Accuracy: 0.84
36
Accuracy: 0.8
37
Accuracy: 0.9
38
Accuracy: 0.76
39
Accuracy: 0.8
40
Accuracy: 0.86
41
Accuracy: 0.8
42
Accuracy: 0.84
43
Accuracy: 0.78
44
Accuracy: 0.74
45
Accuracy: 0.74
46
Accuracy: 0.84
47
Accuracy: 0.8
48
Accuracy: 0.84
49
Accuracy: 0.7
50
Accuracy: 0.82
51
Accuracy: 0.82
52
Accuracy: 0.78
53
Accuracy: 0.78
54
Accuracy: 0.78
55
Accuracy: 0.74
56
Accuracy: 0.68
57
Accuracy:

In this line of code, the parameters dictionary is being defined. It contains the hyperparameters for the linear SVM model that will be passed to the GridSearchCV function.

The hyperparameter C determines the trade-off between achieving a low training error and a low testing error. In this case, C is set to [1], which means the value of C will be searched for only once, i.e., C=1. C is a regularization parameter, and a smaller value of C will result in a wider margin, while a larger value of C will result in a narrower margin but potentially better classification.

    parameters = {'C': [0.1, 0.5, 1, 5, 10, 50, 100]}

This will test the SVM model with C values of 0.1, 0.5, 1, 5, 10, 50, and 100. The GridSearchCV function will search for the best C value among these values, based on the accuracy of the predictions on the test data.

Note that the larger the value of C, the more the model will try to avoid misclassifying any instances in the training set. On the other hand, a smaller value of C will result in a wider margin, and the model will have a higher tolerance for misclassified instances.

### Individual Features

In [7]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                        NonMot1['APPRDX'], test_size=0.2)
        
        svm_model = SVC(kernel='linear')
        parameters = {'C': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100]}
        svm_model = GridSearchCV(svm_model, parameters)
        svm_model.fit(train_data, train_labels)

        predictions = svm_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_svmL.csv", output, delimiter=",")
        joblib.dump(svm_model, f"{col}_svmL_model.joblib")

Accuracy for column Anxiety: 0.52
Accuracy for column Apathy: 0.72
Accuracy for column Benton: 0.62
Accuracy for column Clock: 0.54
Accuracy for column Cognition: 0.56
Accuracy for column COGSTATE: 0.7
Accuracy for column Constipate: 0.6
Accuracy for column Depress: 0.68
Accuracy for column DopaDefic: 0.66
Accuracy for column Education: 0.64
Accuracy for column Epworth: 0.54
Accuracy for column Fatigue: 0.62
Accuracy for column Geriatric_Depression: 0.58
Accuracy for column Hallucination: 0.56
Accuracy for column Hand: 0.56
Accuracy for column Hopkins: 0.48
Accuracy for column Hopkins_Recog: 0.66
Accuracy for column Impulsive_CompulsiveBehavior: 0.62
Accuracy for column Impulsive_ICD: 0.72
Accuracy for column LetterNumber: 0.62
Accuracy for column Lexical_Fluency: 0.6
Accuracy for column LightHead: 0.58
Accuracy for column Modif_Boston: 0.64
Accuracy for column Montreal_Cognitive: 0.7
Accuracy for column Pain: 0.68
Accuracy for column REM_AwakeDream: 0.62
Accuracy for column REM_AwakeP

## SVM Radial

### All Features

In [38]:
output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    svm_model = SVC(kernel='rbf')
    parameters = {'C': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100], 'gamma': [0.01, 0.1, 1, 10]}
    svm_model = GridSearchCV(svm_model, parameters)
    svm_model.fit(train_data, train_labels)
    
    predictions = svm_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100svmRadial.csv", output, delimiter=",")


1
Accuracy: 0.56
2
Accuracy: 0.74
3
Accuracy: 0.66
4
Accuracy: 0.76
5
Accuracy: 0.66
6
Accuracy: 0.58
7
Accuracy: 0.68
8
Accuracy: 0.62
9
Accuracy: 0.68
10
Accuracy: 0.68
11
Accuracy: 0.66
12
Accuracy: 0.62
13
Accuracy: 0.66
14
Accuracy: 0.72
15
Accuracy: 0.64
16
Accuracy: 0.64
17
Accuracy: 0.76
18
Accuracy: 0.7
19
Accuracy: 0.6
20
Accuracy: 0.54
21
Accuracy: 0.72
22
Accuracy: 0.54
23
Accuracy: 0.66
24
Accuracy: 0.74
25
Accuracy: 0.62
26
Accuracy: 0.72
27
Accuracy: 0.64
28
Accuracy: 0.66
29
Accuracy: 0.72
30
Accuracy: 0.68
31
Accuracy: 0.62
32
Accuracy: 0.64
33
Accuracy: 0.6
34
Accuracy: 0.6
35
Accuracy: 0.66
36
Accuracy: 0.62
37
Accuracy: 0.56
38
Accuracy: 0.62
39
Accuracy: 0.66
40
Accuracy: 0.68
41
Accuracy: 0.64
42
Accuracy: 0.64
43
Accuracy: 0.6
44
Accuracy: 0.56
45
Accuracy: 0.64
46
Accuracy: 0.7
47
Accuracy: 0.72
48
Accuracy: 0.66
49
Accuracy: 0.68
50
Accuracy: 0.7
51
Accuracy: 0.72
52
Accuracy: 0.58
53
Accuracy: 0.64
54
Accuracy: 0.66
55
Accuracy: 0.66
56
Accuracy: 0.72
57
Accur

### Individual Features

In [ ]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                        NonMot1['APPRDX'], test_size=0.2)
        
        svm_model = SVC(kernel='rbf')
        parameters = {'C': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100], 'gamma': [0.01, 0.1, 1, 10]}
        svm_model = GridSearchCV(svm_model, parameters)
        svm_model.fit(train_data, train_labels)

        predictions = svm_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_svmR.csv", output, delimiter=",")
        joblib.dump(svm_model, f"{col}_svmR_model.joblib")

## SVM Poly

In [37]:
output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    svm_model = SVC(kernel='poly', degree=3)
    parameters = {'C': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100], 'gamma': [0.01, 0.1, 1, 10]}
    svm_model = GridSearchCV(svm_model, parameters)
    svm_model.fit(train_data, train_labels)
    
    predictions = svm_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100svmPoly.csv", output, delimiter=",")


1
Accuracy: 0.66
2
Accuracy: 0.7
3
Accuracy: 0.78
4
Accuracy: 0.7
5
Accuracy: 0.74
6
Accuracy: 0.62
7
Accuracy: 0.7
8
Accuracy: 0.64
9
Accuracy: 0.78
10
Accuracy: 0.72
11
Accuracy: 0.72
12
Accuracy: 0.7
13
Accuracy: 0.66
14
Accuracy: 0.76
15
Accuracy: 0.68
16
Accuracy: 0.74
17
Accuracy: 0.74
18
Accuracy: 0.66
19
Accuracy: 0.8
20
Accuracy: 0.78
21
Accuracy: 0.76
22
Accuracy: 0.74
23
Accuracy: 0.74
24
Accuracy: 0.78
25
Accuracy: 0.74
26
Accuracy: 0.74
27
Accuracy: 0.72
28
Accuracy: 0.74
29
Accuracy: 0.72
30
Accuracy: 0.76
31
Accuracy: 0.76
32
Accuracy: 0.66
33
Accuracy: 0.72
34
Accuracy: 0.66
35
Accuracy: 0.78
36
Accuracy: 0.66
37
Accuracy: 0.7
38
Accuracy: 0.72
39
Accuracy: 0.72
40
Accuracy: 0.74
41
Accuracy: 0.72
42
Accuracy: 0.76
43
Accuracy: 0.6
44
Accuracy: 0.82
45
Accuracy: 0.74
46
Accuracy: 0.7
47
Accuracy: 0.72
48
Accuracy: 0.64
49
Accuracy: 0.72
50
Accuracy: 0.76
51
Accuracy: 0.76
52
Accuracy: 0.68
53
Accuracy: 0.74
54
Accuracy: 0.68
55
Accuracy: 0.7
56
Accuracy: 0.66
57
Accurac

for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                        NonMot1['APPRDX'], test_size=0.2)
        
        svm_model = SVC(kernel='poly', degree=3)
        parameters = {'C': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100], 'gamma': [0.01, 0.1, 1, 10]}
        svm_model = GridSearchCV(svm_model, parameters)
        svm_model.fit(train_data, train_labels)

        predictions = svm_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_svmP.csv", output, delimiter=",")
        joblib.dump(svm_model, f"{col}_svmP_model.joblib")

    hinge: This is the default loss function for the linear SVM classifier, which maximizes the margin between the two classes. It is also known as the max-margin hinge loss, and is typically used for binary classification problems.

    log_loss: This is the logistic regression loss function, also known as the cross-entropy loss. It is used to train linear classifiers that predict class probabilities. It is suitable for both binary and multi-class classification problems.

    modified_huber: This is a smoothed version of the hinge loss that is less sensitive to outliers. It combines the best properties of the hinge and log loss functions and is useful for noisy data and datasets with mislabeled samples.

    squared_hinge: This is a variant of the hinge loss that is squared. It is more sensitive to samples that are closer to the decision boundary and can be used to train non-linear classifiers.

    perceptron: This is the perceptron algorithm, which is a linear binary classifier. It updates the weight vector in each iteration using only the misclassified samples.

    squared_error: This is the mean squared error (MSE) loss function, which is used to train linear regression models. It minimizes the sum of the squared errors between the predicted and actual target values.

    huber: This is a robust regression loss function that combines the best properties of MSE and absolute error loss functions. It is less sensitive to outliers than the MSE loss and is useful for datasets with noisy or corrupted samples.

    epsilon_insensitive: This is the epsilon-insensitive loss function, which is used for support vector regression (SVR). It ignores errors that are within a certain distance epsilon from the actual target value.

    squared_epsilon_insensitive: This is a squared version of the epsilon-insensitive loss function, which is also used for SVR. It is more sensitive to larger errors than the regular epsilon-insensitive loss.

# Boosting

## Adaptive Boosting

In [32]:

output = []
for i in range(1, 101):
    print(f'Iteration {i}')
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    rf_model = RandomForestClassifier()
    ada_model = AdaBoostClassifier(base_estimator=rf_model)
    parameters = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 1, 10]}
    ada_model = GridSearchCV(ada_model, parameters)
    cv_scores = cross_val_score(ada_model, X, y, cv=10)
    mean_accuracy = np.mean(cv_scores)
    std_accuracy = np.std(cv_scores)
    print(f'Cross-validation accuracy: {mean_accuracy} +/- {std_accuracy}')
    output.append(mean_accuracy)

np.savetxt("NonMotor_unCateg_100AdaBoost.csv", output, delimiter=",")


Iteration 1
Cross-validation accuracy: 0.8360000000000001 +/- 0.04176122603564219
Iteration 2
Cross-validation accuracy: 0.8360000000000001 +/- 0.04882622246293482
Iteration 3
Cross-validation accuracy: 0.8240000000000001 +/- 0.044542114902640165
Iteration 4
Cross-validation accuracy: 0.8200000000000001 +/- 0.0626099033699941
Iteration 5
Cross-validation accuracy: 0.8160000000000001 +/- 0.047999999999999994
Iteration 6
Cross-validation accuracy: 0.8280000000000001 +/- 0.047497368348151665
Iteration 7
Cross-validation accuracy: 0.8360000000000001 +/- 0.060530983801686214
Iteration 8
Cross-validation accuracy: 0.804 +/- 0.07255342858886822
Iteration 9
Cross-validation accuracy: 0.8160000000000001 +/- 0.059866518188383046
Iteration 10
Cross-validation accuracy: 0.8160000000000001 +/- 0.06248199740725322
Iteration 11
Cross-validation accuracy: 0.8160000000000001 +/- 0.059866518188383046
Iteration 12
Cross-validation accuracy: 0.844 +/- 0.07031358332498779
Iteration 13
Cross-validation accu

In [13]:

output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    rf_model = RandomForestClassifier()
    ada_model = AdaBoostClassifier(base_estimator=rf_model)
    parameters = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 1, 10]}
    ada_model = GridSearchCV(ada_model, parameters)
    ada_model.fit(train_data, train_labels)
    
    predictions = ada_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100AdaBoost.csv", output, delimiter=",")


1
Accuracy: 0.84
2
Accuracy: 0.88
3
Accuracy: 0.92
4
Accuracy: 0.78
5
Accuracy: 0.9
6
Accuracy: 0.86
7
Accuracy: 0.82
8
Accuracy: 0.84
9
Accuracy: 0.82
10
Accuracy: 0.78
11
Accuracy: 0.82
12
Accuracy: 0.82
13
Accuracy: 0.76
14
Accuracy: 0.88
15
Accuracy: 0.84
16
Accuracy: 0.76
17
Accuracy: 0.78
18
Accuracy: 0.84
19
Accuracy: 0.92
20
Accuracy: 0.78
21
Accuracy: 0.92
22
Accuracy: 0.8
23
Accuracy: 0.82
24
Accuracy: 0.84
25
Accuracy: 0.8
26
Accuracy: 0.78
27
Accuracy: 0.8
28
Accuracy: 0.86
29
Accuracy: 0.86
30
Accuracy: 0.84
31
Accuracy: 0.72
32
Accuracy: 0.8
33
Accuracy: 0.88
34
Accuracy: 0.86
35
Accuracy: 0.8
36
Accuracy: 0.86
37
Accuracy: 0.84
38
Accuracy: 0.78
39
Accuracy: 0.8
40
Accuracy: 0.84
41
Accuracy: 0.84
42
Accuracy: 0.74
43
Accuracy: 0.78
44
Accuracy: 0.8
45
Accuracy: 0.84
46
Accuracy: 0.86
47
Accuracy: 0.9
48
Accuracy: 0.82
49
Accuracy: 0.8
50
Accuracy: 0.94
51
Accuracy: 0.88
52
Accuracy: 0.8
53
Accuracy: 0.84
54
Accuracy: 0.76
55
Accuracy: 0.84
56
Accuracy: 0.84
57
Accuracy:

In [18]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                                                            NonMot1['APPRDX'], test_size=0.2)
        
        rf_model = RandomForestClassifier()
        ada_model = AdaBoostClassifier(base_estimator=rf_model)
        parameters = {'n_estimators': [300], 'learning_rate': [0.1]}
        ada_model = GridSearchCV(ada_model, parameters, n_jobs=-1)
        ada_model.fit(train_data, train_labels)

        predictions = ada_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_AdaBoost.csv", output, delimiter=",")
        joblib.dump(ada_model, f"{col}_AdaBoost_model.joblib")


Accuracy for column Anxiety: 0.7
Accuracy for column Apathy: 0.7
Accuracy for column Benton: 0.72
Accuracy for column Clock: 0.58
Accuracy for column Cognition: 0.56
Accuracy for column COGSTATE: 0.72
Accuracy for column Constipate: 0.68
Accuracy for column Depress: 0.6
Accuracy for column DopaDefic: 0.58
Accuracy for column Education: 0.66
Accuracy for column Epworth: 0.66
Accuracy for column Fatigue: 0.76
Accuracy for column Geriatric_Depression: 0.66
Accuracy for column Hallucination: 0.64
Accuracy for column Hand: 0.7
Accuracy for column Hopkins: 0.62
Accuracy for column Hopkins_Recog: 0.6
Accuracy for column Impulsive_CompulsiveBehavior: 0.64
Accuracy for column Impulsive_ICD: 0.64
Accuracy for column LetterNumber: 0.64
Accuracy for column Lexical_Fluency: 0.6
Accuracy for column LightHead: 0.68
Accuracy for column Modif_Boston: 0.52
Accuracy for column Montreal_Cognitive: 0.68
Accuracy for column Pain: 0.72
Accuracy for column REM_AwakeDream: 0.6
Accuracy for column REM_AwakeProb

from joblib import Parallel, delayed

def process_column(col):
    output = []
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], NonMot1['APPRDX'], test_size=0.2)
    rf_model = RandomForestClassifier()
    ada_model = AdaBoostClassifier(base_estimator=rf_model)
    parameters = {'n_estimators': [300], 'learning_rate': [0.1]}
    ada_model = GridSearchCV(ada_model, parameters, n_jobs=-1)
    ada_model.fit(train_data, train_labels)
    predictions = ada_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy for column {col}: {accuracy}')
    output.append(accuracy)
    np.savetxt(f"{col}_AdaBoost.csv", output, delimiter=",")
    joblib.dump(ada_model, f"{col}_AdaBoost_model.joblib")

Parallel(n_jobs=-1)(delayed(process_column)(col) for col in NonMot1.columns if col != 'APPRDX')


## Gradient Boosting

In [39]:

output = []
for i in range(1, 101):
    print(f'Iteration {i}')
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    gb_model = GradientBoostingClassifier()
    parameters = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 1, 10], 'max_depth': [3, 4, 5]}
    gb_model = GridSearchCV(gb_model, parameters)
    cv_scores = cross_val_score(gb_model, X, y, cv=10)
    mean_accuracy = np.mean(cv_scores)
    std_accuracy = np.std(cv_scores)
    print(f'Cross-validation accuracy: {mean_accuracy} +/- {std_accuracy}')
    output.append(mean_accuracy)

np.savetxt("NonMotor_unCateg_100GradientBoost.csv", output, delimiter=",")


Iteration 1
Cross-validation accuracy: 0.8160000000000001 +/- 0.06740919818541086
Iteration 2
Cross-validation accuracy: 0.808 +/- 0.07547184905645282
Iteration 3
Cross-validation accuracy: 0.808 +/- 0.07110555533852471
Iteration 4
Cross-validation accuracy: 0.8200000000000001 +/- 0.06511528238439881
Iteration 5
Cross-validation accuracy: 0.8160000000000001 +/- 0.06740919818541086
Iteration 6
Cross-validation accuracy: 0.8200000000000001 +/- 0.06985699678629191
Iteration 7
Cross-validation accuracy: 0.82 +/- 0.0626099033699941
Iteration 8
Cross-validation accuracy: 0.8160000000000001 +/- 0.06740919818541086
Iteration 9
Cross-validation accuracy: 0.8160000000000001 +/- 0.06740919818541086
Iteration 10
Cross-validation accuracy: 0.8160000000000001 +/- 0.08616263691415205
Iteration 11
Cross-validation accuracy: 0.8240000000000001 +/- 0.06740919818541084
Iteration 12
Cross-validation accuracy: 0.82 +/- 0.0698569967862919
Iteration 13
Cross-validation accuracy: 0.836 +/- 0.07031358332498774

In [15]:

output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    gb_model = GradientBoostingClassifier()   
    parameters = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 1, 10]}
    gb_model = GridSearchCV(gb_model, parameters)
    gb_model.fit(train_data, train_labels)
    
    predictions = gb_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100GradientBoosting.csv", output, delimiter=",")


1
Accuracy: 0.92
2
Accuracy: 0.8
3
Accuracy: 0.86
4
Accuracy: 0.92
5
Accuracy: 0.8
6
Accuracy: 0.84
7
Accuracy: 0.84
8
Accuracy: 0.76
9
Accuracy: 0.92
10
Accuracy: 0.82
11
Accuracy: 0.92
12
Accuracy: 0.8
13
Accuracy: 0.76
14
Accuracy: 0.8
15
Accuracy: 0.76
16
Accuracy: 0.8
17
Accuracy: 0.9
18
Accuracy: 0.84
19
Accuracy: 0.88
20
Accuracy: 0.82
21
Accuracy: 0.86
22
Accuracy: 0.76
23
Accuracy: 0.86
24
Accuracy: 0.86
25
Accuracy: 0.86
26
Accuracy: 0.82
27
Accuracy: 0.8
28
Accuracy: 0.88
29
Accuracy: 0.74
30
Accuracy: 0.86
31
Accuracy: 0.8
32
Accuracy: 0.84
33
Accuracy: 0.86
34
Accuracy: 0.9
35
Accuracy: 0.84
36
Accuracy: 0.86
37
Accuracy: 0.88
38
Accuracy: 0.86
39
Accuracy: 0.74
40
Accuracy: 0.82
41
Accuracy: 0.86
42
Accuracy: 0.84
43
Accuracy: 0.88
44
Accuracy: 0.84
45
Accuracy: 0.82
46
Accuracy: 0.88
47
Accuracy: 0.82
48
Accuracy: 0.84
49
Accuracy: 0.86
50
Accuracy: 0.86
51
Accuracy: 0.88
52
Accuracy: 0.82
53
Accuracy: 0.82
54
Accuracy: 0.86
55
Accuracy: 0.82
56
Accuracy: 0.88
57
Accurac

In [ ]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                                                NonMot1['APPRDX'], test_size=0.2)

        rf_model = RandomForestClassifier()
        gb_model = GradientBoostingClassifier(base_estimator=rf_model)
        gb_model = GradientBoostingClassifier()
        parameters = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 1, 10]}
        gb_model = GridSearchCV(gb_model, parameters)
        gb_model.fit(train_data, train_labels)
        
        predictions = gb_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_GradBoost.csv", output, delimiter=",")
        joblib.dump(gb_model, f"{col}_GradBoost_model.joblib")


## Extra Gradient Boosting

In [4]:
NonMot1['APPRDX'] = NonMot1['APPRDX'].replace(['Patient'], 0)
NonMot1['APPRDX'] = NonMot1['APPRDX'].replace(['Healthy'], 1)
NonMot1.head(2)

,APPRDX,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,0,1,0,12.16,7,1,1,1,1,0,...,57,1,0,45,59,47.5,110,204,17,1
1,0,1,1,7.76,6,1,1,0,0,0,...,36,1,3,40,39,52.0,27,52,9,0


In [5]:
import xgboost as xgb

output = []
for i in range(1, 101):
    print(f'Iteration {i}')
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    xgb_model = xgb.XGBClassifier()
    parameters = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 1, 10], 'max_depth': [3, 4, 5]}
    xgb_model = GridSearchCV(xgb_model, parameters)
    cv_scores = cross_val_score(xgb_model, X, y, cv=10)
    mean_accuracy = np.mean(cv_scores)
    std_accuracy = np.std(cv_scores)
    print(f'Cross-validation accuracy: {mean_accuracy} +/- {std_accuracy}')
    output.append(mean_accuracy)

np.savetxt("NonMotor_unCateg_100XGBClassifier.csv", output, delimiter=",")


Iteration 1
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 2
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 3
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 4
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 5
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 6
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 7
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 8
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 9
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 10
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 11
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 12
Cross-validation accuracy: 0.8320000000000001 +/- 0.06645299090334458
Iteration 13


KeyboardInterrupt: 

In [9]:

output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    xgb_model = XGBClassifier()
    parameters = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 1, 10]}
    xgb_model = GridSearchCV(xgb_model, parameters)
    xgb_model.fit(train_data, train_labels)
    
    predictions = xgb_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100XGBoost.csv", output, delimiter=",")


1
Accuracy: 0.88
2
Accuracy: 0.8
3
Accuracy: 0.84
4
Accuracy: 0.86
5
Accuracy: 0.88
6
Accuracy: 0.88
7
Accuracy: 0.82
8
Accuracy: 0.78
9
Accuracy: 0.86
10
Accuracy: 0.82
11
Accuracy: 0.82
12
Accuracy: 0.8
13
Accuracy: 0.92
14
Accuracy: 0.78
15
Accuracy: 0.82
16
Accuracy: 0.9
17
Accuracy: 0.8
18
Accuracy: 0.9
19
Accuracy: 0.9
20
Accuracy: 0.86
21
Accuracy: 0.82
22
Accuracy: 0.82
23
Accuracy: 0.82
24
Accuracy: 0.92
25
Accuracy: 0.92
26
Accuracy: 0.8
27
Accuracy: 0.84
28
Accuracy: 0.86
29
Accuracy: 0.86
30
Accuracy: 0.78
31
Accuracy: 0.86
32
Accuracy: 0.86
33
Accuracy: 0.82
34
Accuracy: 0.8
35
Accuracy: 0.86
36
Accuracy: 0.8
37
Accuracy: 0.78
38
Accuracy: 0.82
39
Accuracy: 0.84
40
Accuracy: 0.8
41
Accuracy: 0.78
42
Accuracy: 0.78
43
Accuracy: 0.9
44
Accuracy: 0.74
45
Accuracy: 0.74
46
Accuracy: 0.86
47
Accuracy: 0.82
48
Accuracy: 0.78
49
Accuracy: 0.9
50
Accuracy: 0.78
51
Accuracy: 0.74
52
Accuracy: 0.82
53
Accuracy: 0.72
54
Accuracy: 0.86
55
Accuracy: 0.76
56
Accuracy: 0.82
57
Accuracy: 

In [ ]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                                                NonMot1['APPRDX'], test_size=0.2)
        
        xgb_model = XGBClassifier()
        parameters = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 1, 10]}
        xgb_model = GridSearchCV(xgb_model, parameters)
        xgb_model.fit(train_data, train_labels)
        
        predictions = xgb_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_XGBoost.csv", output, delimiter=",")
        joblib.dump(xgb_model, f"{col}_XGBoost_model.joblib")


# LDA

In [7]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

output = []
for i in range(1, 101):
    print(i)
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    lda_model = LinearDiscriminantAnalysis(solver='svd', n_components=None)
    lda_model.fit(X_train, y_train)
    
    cv_scores = cross_val_score(lda_model, X, y, cv=10)
    mean_cv_score = np.mean(cv_scores)
    print(f'Mean cross-validation accuracy: {mean_cv_score}')
    
    predictions = lda_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'Test accuracy: {accuracy}')
    output.append(accuracy)
    
np.savetxt("NonMotor_unCateg_100LDA.csv", output, delimiter=",")


1
Mean cross-validation accuracy: 0.776
Test accuracy: 0.8
2
Mean cross-validation accuracy: 0.776
Test accuracy: 0.72
3
Mean cross-validation accuracy: 0.776
Test accuracy: 0.74
4
Mean cross-validation accuracy: 0.776
Test accuracy: 0.88
5
Mean cross-validation accuracy: 0.776
Test accuracy: 0.82
6
Mean cross-validation accuracy: 0.776
Test accuracy: 0.82
7
Mean cross-validation accuracy: 0.776
Test accuracy: 0.68
8
Mean cross-validation accuracy: 0.776
Test accuracy: 0.76
9
Mean cross-validation accuracy: 0.776
Test accuracy: 0.84
10
Mean cross-validation accuracy: 0.776
Test accuracy: 0.78
11
Mean cross-validation accuracy: 0.776
Test accuracy: 0.68
12
Mean cross-validation accuracy: 0.776
Test accuracy: 0.82
13
Mean cross-validation accuracy: 0.776
Test accuracy: 0.8
14
Mean cross-validation accuracy: 0.776
Test accuracy: 0.74
15
Mean cross-validation accuracy: 0.776
Test accuracy: 0.78
16
Mean cross-validation accuracy: 0.776
Test accuracy: 0.78
17
Mean cross-validation accuracy: 

    solver: LDA has four solvers to choose from, namely svd, lsqr, eigen, and lsqr. The default solver is svd, which is a good choice for small datasets. If you have a larger dataset, you may want to try the lsqr or eigen solvers.
    shrinkage: LDA can use shrinkage to improve the estimation of the covariance matrix of the data. Shrinkage can be particularly useful when the number of features is much larger than the number of training samples. The shrinkage parameter can be set to auto to automatically choose the shrinkage intensity, or a float value between 0 and 1 to specify a specific level of shrinkage.
    n_components: LDA can also be used for dimensionality reduction, in which case you would set the n_components parameter to a value smaller than the original number of features. This can help to reduce overfitting and speed up training and prediction. However, for classification purposes, you would typically set n_components to None to retain all the features.
    
Here's a brief explanation of the four solvers available in scikit-learn's LDA implementation:

    svd: This is the default solver and is based on singular value decomposition (SVD). It is the most efficient solver for small to moderate-sized datasets, and it works well when the number of samples is much larger than the number of features.

    lsqr: This solver is based on a conjugate gradient algorithm, and it is particularly effective for larger datasets. It works by iteratively solving the linear system Ax = b, where A is the input data, b is the target variable, and x is the projection matrix.

    eigen: This solver computes the eigenvalues and eigenvectors of the covariance matrix of the input data. It is a good choice when the number of features is much smaller than the number of samples.

    eigen-sparse: This is a variation of the eigen solver that is designed for sparse input data.

# QDA

In [8]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

output = []
for i in range(1, 101):
    print(i)
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    qda_model = QuadraticDiscriminantAnalysis()
    qda_model.fit(X_train, y_train)
    
    cv_scores = cross_val_score(qda_model, X, y, cv=10)
    mean_cv_score = np.mean(cv_scores)
    print(f'Mean cross-validation accuracy: {mean_cv_score}')
    
    predictions = qda_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'Test accuracy: {accuracy}')
    output.append(accuracy)
    
np.savetxt("NonMotor_unCateg_100QDA.csv", output, delimiter=",")


1
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.7
2


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.6
3
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.58
4
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.7
5
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.66
6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-

Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.68
7
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.72
8
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.8
9
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.58
10


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-

Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.8
93
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.78
94
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.72
95
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.66
96


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-

Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.74
97
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.66
98
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.7
99
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.5
100
Mean cross-validation accuracy: 0.6799999999999999
Test accuracy: 0.76


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-

# Neural Network

In [12]:

output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    mlp_model = MLPClassifier()
    parameters = {'hidden_layer_sizes': [(20,10,5), (12,8)], 
                  'alpha': [0.0001, 0.001, 0.01, 0.1], 
                  'max_iter': [5000], 
                  'activation': ['relu', 'logistic']}
    mlp_model = GridSearchCV(mlp_model, parameters)
    mlp_model.fit(train_data, train_labels)
    
    predictions = mlp_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100MLPNeuralNet.csv", output, delimiter=",")


1
Accuracy: 0.74
2
Accuracy: 0.78
3
Accuracy: 0.76
4
Accuracy: 0.8
5
Accuracy: 0.8
6
Accuracy: 0.8
7
Accuracy: 0.74
8
Accuracy: 0.74
9
Accuracy: 0.76
10
Accuracy: 0.78
11
Accuracy: 0.68
12
Accuracy: 0.68
13
Accuracy: 0.92
14
Accuracy: 0.76
15
Accuracy: 0.8
16
Accuracy: 0.82
17
Accuracy: 0.7
18
Accuracy: 0.76
19
Accuracy: 0.74
20
Accuracy: 0.72
21
Accuracy: 0.82
22
Accuracy: 0.84
23
Accuracy: 0.8
24
Accuracy: 0.8
25
Accuracy: 0.76
26
Accuracy: 0.74
27
Accuracy: 0.7
28
Accuracy: 0.82
29
Accuracy: 0.78
30
Accuracy: 0.8
31
Accuracy: 0.78
32
Accuracy: 0.76
33
Accuracy: 0.86
34
Accuracy: 0.78
35
Accuracy: 0.74
36
Accuracy: 0.82
37
Accuracy: 0.78
38
Accuracy: 0.76
39
Accuracy: 0.82
40
Accuracy: 0.88
41
Accuracy: 0.76
42
Accuracy: 0.82
43
Accuracy: 0.58
44
Accuracy: 0.72
45
Accuracy: 0.78
46
Accuracy: 0.88
47
Accuracy: 0.8
48
Accuracy: 0.7
49
Accuracy: 0.78
50
Accuracy: 0.76
51
Accuracy: 0.76
52
Accuracy: 0.78
53
Accuracy: 0.76
54
Accuracy: 0.82
55
Accuracy: 0.74
56
Accuracy: 0.72
57
Accuracy:

In [ ]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                                                NonMot1['APPRDX'], test_size=0.2)
        
        mlp_model = MLPClassifier()
        parameters = {'hidden_layer_sizes': [(10,), (5,), (10,50), (10,6,2)], 
                  'alpha': [0.0001, 0.001, 0.01, 0.1], 
                  'max_iter': [2000], 
                  'activation': ['relu', 'logistic']}
        mlp_model = GridSearchCV(mlp_model, parameters)
        mlp_model.fit(train_data, train_labels)
        
        predictions = mlp_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_MLPNN.csv", output, delimiter=",")
        joblib.dump(mlp_model, f"{col}_MLPNN_model.joblib")


# KNN

In [11]:
from sklearn.metrics import accuracy_score 

output = []
for i in range(1, 101):
    print(i)
    X = NonMot1.drop('APPRDX', axis=1)
    y = NonMot1['APPRDX']
    
    # Splitting the dataset into the Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Training the K-NN model on the Training set
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)

    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix, accuracy_score
    cm = confusion_matrix(y_test, y_pred)
    ac = accuracy_score(y_test, y_pred)
    print(cm)
    print(ac)
    output.append(ac)

np.savetxt("NonMotor_unCateg_100KNN.csv", output, delimiter=",")


1
[[11  3]
 [11 25]]
0.72
2
[[ 7  7]
 [ 9 27]]
0.68
3
[[ 8 13]
 [ 4 25]]
0.66
4
[[ 8  5]
 [12 25]]
0.66
5
[[12 11]
 [ 6 21]]
0.66
6
[[ 8  6]
 [12 24]]
0.64
7
[[12  2]
 [ 9 27]]
0.78
8
[[14  5]
 [ 4 27]]
0.82
9
[[ 8  6]
 [ 5 31]]
0.78
10
[[10  9]
 [ 4 27]]
0.74
11
[[11  4]
 [ 7 28]]
0.78
12
[[ 8 10]
 [ 7 25]]
0.66
13
[[ 9  5]
 [12 24]]
0.66
14
[[13  8]
 [ 5 24]]
0.74
15
[[11  3]
 [12 24]]
0.7
16
[[16  3]
 [ 6 25]]
0.82
17
[[ 9  4]
 [18 19]]
0.56
18
[[11  5]
 [ 5 29]]
0.8
19
[[12  8]
 [ 5 25]]
0.74
20
[[ 9  7]
 [10 24]]
0.66
21
[[10  3]
 [ 8 29]]
0.78
22


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

[[ 9  3]
 [10 28]]
0.74
23
[[ 8  7]
 [ 5 30]]
0.76
24
[[10  4]
 [ 9 27]]
0.74
25
[[15  7]
 [ 7 21]]
0.72
26
[[11  8]
 [ 4 27]]
0.76
27
[[ 8 10]
 [ 9 23]]
0.62
28
[[13  7]
 [ 6 24]]
0.74
29
[[ 9  8]
 [ 9 24]]
0.66
30
[[14  4]
 [ 7 25]]
0.78
31
[[ 6 10]
 [ 5 29]]
0.7
32
[[12  8]
 [ 1 29]]
0.82
33
[[ 7  8]
 [ 6 29]]
0.72
34
[[11  1]
 [11 27]]
0.76
35
[[11  7]
 [14 18]]
0.58
36
[[10  6]
 [10 24]]
0.68
37
[[ 9  6]
 [ 8 27]]
0.72
38
[[ 9  7]
 [ 6 28]]
0.74
39
[[10  7]
 [ 5 28]]
0.76
40
[[11  6]
 [ 2 31]]
0.84
41
[[11  3]
 [10 26]]
0.74
42
[[10  2]
 [11 27]]
0.74
43
[[ 7  8]
 [ 9 26]]
0.66
44
[[10  5]
 [ 8 27]]
0.74
45
[[10  2]
 [ 7 31]]
0.82
46
[[10 10]
 [ 3 27]]
0.74
47
[[11  7]
 [ 5 27]]
0.76
48


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

[[10  8]
 [ 5 27]]
0.74
49
[[11  7]
 [ 9 23]]
0.68
50
[[14  5]
 [ 6 25]]
0.78
51
[[11  4]
 [10 25]]
0.72
52
[[13  6]
 [ 7 24]]
0.74
53
[[12  7]
 [ 7 24]]
0.72
54
[[12  6]
 [ 5 27]]
0.78
55
[[12  4]
 [13 21]]
0.66
56
[[ 9  4]
 [ 8 29]]
0.76
57
[[11  7]
 [ 7 25]]
0.72
58
[[11  7]
 [ 8 24]]
0.7
59
[[ 9  7]
 [ 9 25]]
0.68
60
[[ 9  8]
 [10 23]]
0.64
61
[[14  5]
 [ 4 27]]
0.82
62
[[11  3]
 [16 20]]
0.62
63
[[ 5 14]
 [ 5 26]]
0.62
64
[[14  2]
 [10 24]]
0.76
65
[[ 7  7]
 [12 24]]
0.62
66
[[12  7]
 [ 6 25]]
0.74
67
[[14  6]
 [ 2 28]]
0.84
68
[[10 10]
 [ 5 25]]
0.7
69
[[ 9  3]
 [12 26]]
0.7
70
[[10  7]
 [ 4 29]]
0.78
71
[[12  7]
 [ 5 26]]
0.76
72
[[ 7  9]
 [ 9 25]]
0.64
73
[[ 8  9]
 [ 5 28]]
0.72
74


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

[[11  5]
 [10 24]]
0.7
75
[[ 9  9]
 [ 9 23]]
0.64
76
[[12  3]
 [ 7 28]]
0.8
77
[[16  4]
 [ 8 22]]
0.76
78
[[ 8 15]
 [ 7 20]]
0.56
79
[[11  7]
 [10 22]]
0.66
80
[[11  5]
 [ 9 25]]
0.72
81
[[11  9]
 [ 6 24]]
0.7
82
[[12 11]
 [ 5 22]]
0.68
83
[[ 9  8]
 [ 7 26]]
0.7
84
[[11  9]
 [ 6 24]]
0.7
85
[[ 8 16]
 [ 2 24]]
0.64
86
[[11  4]
 [ 5 30]]
0.82
87
[[17  6]
 [ 4 23]]
0.8
88
[[14  5]
 [ 3 28]]
0.84
89
[[ 9  7]
 [ 9 25]]
0.68
90
[[10  8]
 [ 4 28]]
0.76
91
[[11  7]
 [13 19]]
0.6
92
[[11  6]
 [ 7 26]]
0.74
93
[[10  6]
 [19 15]]
0.5
94
[[ 6  9]
 [ 3 32]]
0.76
95
[[11  9]
 [ 9 21]]
0.64
96
[[10  6]
 [ 7 27]]
0.74
97
[[12  8]
 [ 4 26]]
0.76
98
[[11  7]
 [ 5 27]]
0.76
99
[[ 9  8]
 [ 6 27]]
0.72
100
[[13  6]
 [ 7 24]]
0.74


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim


#scipy.stats.mode(a, axis=0, nan_policy='propagate', keepdims=None)
output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    knn_model = KNeighborsClassifier()
    parameters = {'n_neighbors': [5, 10, 15, 20]}
    knn_model = GridSearchCV(knn_model, parameters)
    knn_model.fit(train_data, train_labels)
    
    predictions = knn_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100KNN.csv", output, delimiter=",")


In [ ]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], 
                                                                NonMot1['APPRDX'], test_size=0.2)
        
        knn_model = KNeighborsClassifier()
        parameters = {'n_neighbors': [5, 10, 15, 20]}
        knn_model = GridSearchCV(knn_model, parameters)
        knn_model.fit(train_data, train_labels)
        
        predictions = knn_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_KNN.csv", output, delimiter=",")
        joblib.dump(knn_model, f"{col}_KNN_model.joblib")


# Voting and Stacking

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

# Load the dataset and split into training and testing sets
X, y = NonMot1.drop('APPRDX', axis=1), NonMot1['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the base models
base_models = [('rf', RandomForestClassifier(n_estimators=100, random_state=1)),
               ('ada', AdaBoostClassifier(n_estimators=100, random_state=1)),
               ('gb', GradientBoostingClassifier(n_estimators=100, random_state=1)),
               ('svc', SVC(probability=True, random_state=1)),
               ('knn', KNeighborsClassifier(n_neighbors=5))]

# Define the voting ensemble model
voting_model = VotingClassifier(estimators=base_models)

# Define the stacking ensemble model
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Train and evaluate the voting ensemble model
voting_model.fit(X_train, y_train)
voting_predictions = voting_model.predict(X_test)
voting_accuracy = accuracy_score(y_test, voting_predictions)
print(f'Voting ensemble accuracy: {voting_accuracy:.4f}')

# Train and evaluate the stacking ensemble model
stacking_model.fit(X_train, y_train)
stacking_predictions = stacking_model.predict(X_test)
stacking_accuracy = accuracy_score(y_test, stacking_predictions)
print(f'Stacking ensemble accuracy: {stacking_accuracy:.4f}')


Voting and stacking are two popular ensemble learning techniques in machine learning. They are used to combine the predictions of multiple base models to improve the accuracy and generalization of the final model.

Voting: In a voting ensemble, multiple base models are trained on the same training data and their predictions are combined using majority voting. That is, the final prediction is determined by the most frequently predicted class across all of the base models. Voting can be done either by hard voting or soft voting. In hard voting, the class that gets the most votes across all models is the predicted class, while in soft voting, the probabilities of all models are averaged to determine the final prediction. Voting is a simple and effective ensemble technique, but it works best when the base models have low correlation with each other.

Stacking: In a stacking ensemble, multiple base models are trained on the same training data and their predictions are used as input to a meta-model, which makes the final prediction. The meta-model is trained on the output of the base models, rather than the original input features. The idea behind stacking is to capture the strengths and weaknesses of the base models and learn how to best combine them. Stacking can be done using different meta-models, such as linear regression, logistic regression, or neural networks. The choice of meta-model depends on the problem domain and the nature of the base models. Stacking is more complex than voting, but it can lead to better performance if done correctly.

Overall, both voting and stacking are useful ensemble techniques that can improve the performance of machine learning models. They are particularly useful when the base models have complementary strengths and weaknesses, and when there is a risk of overfitting to the training data.